# Experiment

- run linear program with NEM trading price + NEM marginal carbon intensity
- optimize for prices and carbon + the space inbetween

Before running this notebook, need to run `$ make expt-space-between` in the `energy-py-linear` folder.

## Dataset Preparation

In [ ]:
!pip install pandas -Uq
from pathlib import Path
import pandas as pd


def load_nem_data(table, region=None, region_col='REGIONID'):
    anchor = Path.home() / 'nem-data' / 'data' / table
    clean = anchor.glob('**/clean.parquet')
    fis = [pd.read_parquet(f) for f in clean]
    data = pd.concat(fis, axis=0)

    if region:
        mask = data[region_col] == region
        data = data.loc[mask, :]
    return data

In [ ]:
region = 'SA1'
prices = load_nem_data('TRADINGPRICE', region)
prices = prices.drop_duplicates().set_index('interval-start')
prices = prices.resample('5T').ffill()
prices.head(2)

In [ ]:
sites = pd.read_csv("http://www.nemweb.com.au/Reports/CURRENT/CDEII/CO2EII_AVAILABLE_GENERATORS.CSV", skiprows=1).iloc[:-1, :]
sites.head(2)

## Carbon Data

Carbon data needs a bit of work - select only ENOF rows:

In [ ]:
carbon = load_nem_data('nemde', region, 'RegionID')

In [ ]:
carbon['PeriodID'] = pd.to_datetime(carbon['PeriodID'])
carbon['PeriodID'] = carbon['PeriodID'].dt.tz_localize(None)
carbon

In [ ]:
mask = carbon['Market'] == 'Energy'
carbon = carbon.loc[mask, :]

mask = carbon['DispatchedMarket'] == 'ENOF'
carbon = carbon.loc[mask, :]

carbon['PeriodID'] = pd.to_datetime(carbon['PeriodID'])

Want to join on NEM Sites

In [ ]:
sites.head(2)

In [ ]:
carbon.head(2)

In [ ]:
carbon = carbon.merge(sites[['DUID', 'REGIONID', 'CO2E_EMISSIONS_FACTOR']], left_on='Unit', right_on='DUID', how='left')

In [ ]:
mask = carbon['CO2E_EMISSIONS_FACTOR'].isnull()
carbon = carbon.loc[~mask, :]
assert carbon.isnull().sum().sum() == 0

In [ ]:
carbon['carbon_generation_[tc/h]'] = carbon['CO2E_EMISSIONS_FACTOR'] * carbon['Increase']

carbon = carbon.set_index('PeriodID')

In [ ]:
grp = carbon.groupby(carbon.index).agg({
    'carbon_generation_[tc/h]': 'sum',
    'Increase': 'sum',
})

In [ ]:
grp.sort_values('Increase').head(4)

In [ ]:
carbon.head(2)

Drop wind for coal:

In [ ]:
carbon.loc[
    '2014-01-01 12:00:00', :
]

In [ ]:
ds = prices.merge(carbon, how='left', left_index=True, right_index=True).sort_index()
print(ds.columns)

ds = ds.loc[:,[
    'RRP', 'Increase'
]]
ds.head(3)

In [ ]:
ds = ds.dropna(axis=0)
assert ds.isnull().sum().sum() == 0

In [ ]:
ds.shape

## Battery Modelling

In [ ]:
!pip install -q pandas==1.3.3 fastparquet
import energypylinear as epl

In [ ]:
mdl = epl.Battery(power=2, capacity=4)
mdl

Want to see profit versus carbon per year

Also look at correlation between price & average carbon

In [ ]:
subset = ds.iloc[:1000, :]

opt_price = mdl.optimize(
    prices=subset['RRP'], carbon=subset['Increase'], objective='price'
)

In [ ]:
carbon_opt = mdl.optimize(
    prices=subset['RRP'], carbon=subset['Increase'], objective='carbon'
)

In [ ]:
opt_price = pd.DataFrame(opt_price)
cols = [c for c in opt_price.columns if 'Cost' in c]
opt_price.sum()[cols]

In [ ]:
pd.DataFrame(carbon_opt).sum()[cols]

In [ ]:
ds.head(3)

## Now the full experiment

Lets parallelize this

In [ ]:
def make_summary(data, name, year_month):
    data = pd.DataFrame(data)
    cols = [c for c in data.columns if 'Cost' in c]
    data = pd.DataFrame(data).agg(
        {c: 'sum' for c in cols}
    ).to_frame().T
    data['objective'] = name
    data['year'] = year_month[0]
    data['month'] = year_month[1]
    data['day'] = 1
    return data

mdl = epl.Battery(power=2, capacity=4)

results = []
i = 0
lim = 12 * 8
for year_month, subset in ds.groupby([ds.index.year, ds.index.month]):
    print(year_month)
    price = mdl.optimize(
        prices=subset['RRP'], carbon=subset['Increase'], objective='price', verbose=False
    )
    
    carbon = mdl.optimize(
        prices=subset['RRP'], carbon=subset['Increase'], objective='carbon', verbose=False
    )

    price = make_summary(price, 'price', year_month)
    carbon = make_summary(carbon, 'carbon', year_month)
    res = pd.concat([price, carbon], axis=0)

    (Path.cwd() / 'results').mkdir(exist_ok=True)
    res.to_parquet(f'./results/{year_month[0]}-{year_month[1]}-1.parquet')
    results.append(res)
    
    i += 1
    if i >= lim:
        break
        
results = pd.concat(results, axis=0)
results['date'] = pd.to_datetime(results[['year', 'month', 'day']]).astype(str)
results.to_parquet('./results/monthly-results.parquet')